# Cluster the neighborhoods in the city of Toronto


Installing neccessary packages

In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


#### Step 01. Importing the packages

In [71]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sklearn.cluster import KMeans # for K-means clustering

#### Step 02. Requesting data from the Wikipidea site

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [6]:
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

#### Step 03. Extract the wikitable with neighborhood data

In [7]:
wiki_table = soup.find('table')
#print(wiki_table.prettify())

Importing wikipedia table data into pandas data frame for further processing.

In [8]:
table_rows = wiki_table.find_all('tr')
wiki_output=[]


for tr in table_rows:
    td=tr.find_all('td')
    wiki_data=[]
    for column in td:
        wiki_data.append(column.text.strip())
    wiki_output.append(wiki_data)

wiki_pd=pd.DataFrame(wiki_output, columns =['PostalCode','Borough','Neighborhood'])    

wiki_pd=wiki_pd.drop(wiki_pd.index[0])

wiki_pd = wiki_pd.replace('\n','', regex=True)

wiki_pd.head()



,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


In [9]:
wiki_pd.shape

(180, 3)

#### Step 04. Data Cleaning

In [10]:
#Removing all the "Not assigned" Boroughs
wiki_pd = wiki_pd[wiki_pd.Borough !='Not assigned']

In [11]:
wiki_pd.shape

(103, 3)

In [12]:
wiki_pd = wiki_pd.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [13]:
#replacing / with ,
wiki_pd = wiki_pd.replace('/',',', regex=True)

In [14]:
wiki_pd.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
wiki_pd['Neighborhood'].isna().sum()

0

Assign value in Borough to the Neighborhoods with emplty values

In [16]:
for index, row in wiki_pd.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']


In [17]:
wiki_pd.shape

(103, 3)

#### Step 05. Finding the Lattitude and Longitude using Google Geocoder - csv file

In [18]:
cd

/Users/darshana


In [19]:
cd /Users/darshana/Documents/GitHub

/Users/darshana/Documents/GitHub


In [21]:
geo_pd=pd.read_csv ('Geospatial_Coordinates.csv')

In [24]:
geo_pd.columns=['PostalCode','Latitude','Longitude']

In [25]:
geo_pd.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
wiki_pd.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Merging Lattitude and Longitude using the Postal code column.

In [26]:
wikifull_pd=pd.merge(wiki_pd,geo_pd, on='PostalCode')

In [48]:
wikifull_pd.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Neighborhood Analysis

In [28]:
toronto = wikifull_pd

In [67]:
from geopy.geocoders import Nominatim
import folium 
import requests # to handle requests
from pandas.io.json import json_normalize #to transform JSON files into pandas data frames
import numpy as np

In [29]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto['Borough'].unique()),
        toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### 01. Retreving the Lattitude and Longitude for Totonto using Geopy library

In [31]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [35]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(toronto, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 02. selecting "Scarborough" Borough for neighborhood analysis

In [38]:
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


### Selecting the Boroughs that contain the word "Toronto"

In [144]:
Toronto_data = toronto[toronto['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [145]:
# create map of selected borough using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Scarborough_data['Neighborhood']):
    label = '{}, {}'.format(Toronto_data, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### 03. Explorer the neighborhood using Forsqure

In [146]:
CLIENT_ID = 'V21UD1YVJ4U1MLPYB3HUEFRJJ1PA011WOA3VNLBNF0ZBM01L' # your Foursquare ID
CLIENT_SECRET = '0UF2O5ZQBERLPLQFESLXCIY3PP4A3HACWNUT5X2K5H3SKAWN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: V21UD1YVJ4U1MLPYB3HUEFRJJ1PA011WOA3VNLBNF0ZBM01L
CLIENT_SECRET:0UF2O5ZQBERLPLQFESLXCIY3PP4A3HACWNUT5X2K5H3SKAWN


In [147]:
Toronto_data.loc[0, 'Neighborhood']

'The Beaches'

In [148]:
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [149]:
# Finding the top 100 venues in the area.

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=V21UD1YVJ4U1MLPYB3HUEFRJJ1PA011WOA3VNLBNF0ZBM01L&client_secret=0UF2O5ZQBERLPLQFESLXCIY3PP4A3HACWNUT5X2K5H3SKAWN&v=20180604&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [52]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9711c160ba08001b1c5de7'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [150]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [151]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,T Hamilton & Son Roofing Inc,Construction & Landscaping,43.807985,-79.198194


In [154]:
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4bb6b9446edc76b0d771311c',
   'name': 'Wendy’s',
   'location': {'crossStreet': 'Morningside & Sheppard',
    'lat': 43.80744841934756,
    'lng': -79.19905558052072,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.80744841934756,
      'lng': -79.19905558052072}],
    'distance': 387,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d16e941735',
     'name': 'Fast Food Restaurant',
     'pluralName': 'Fast Food Restaurants',
     'shortName': 'Fast Food',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4bb6b9446edc76b0d771311c-0'},
 {'reasons':

In [140]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


#### 04. Getting all the nearby Neighborhoods 

In [141]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [156]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
Roselawn
Forest Hill North & West
The Annex , North Midtown , Yorkville
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Stn A PO Boxes
First Canadian Place , Underground city
Christie
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , 

In [157]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22
Business reply mail Processing CentrE,17,17,17,17,17,17
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",18,18,18,18,18,18
Central Bay Street,64,64,64,64,64,64
Christie,18,18,18,18,18,18
Church and Wellesley,75,75,75,75,75,75
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [163]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,...,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West , Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [164]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,...,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"India Bazaar , The Beaches West",0.000000,0.000000,0.000000,0.05,0.050000,0.050000,0.00,0.100000,0.000000,...,0.000000,0.000000,0.050000,0.05,0.050000,0.000000,0.000000,0.000000,0.050000,0.05
1,Studio District,0.000000,0.041667,0.000000,0.00,0.000000,0.041667,0.00,0.041667,0.041667,...,0.041667,0.000000,0.041667,0.00,0.000000,0.000000,0.000000,0.041667,0.000000,0.00
2,The Beaches,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.25,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"The Danforth West , Riverdale",0.023256,0.000000,0.023256,0.00,0.069767,0.046512,0.00,0.023256,0.000000,...,0.046512,0.023256,0.000000,0.00,0.046512,0.023256,0.023256,0.000000,0.023256,0.00


In [165]:
#sorting the venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


#adding the data into a dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"India Bazaar , The Beaches West",Breakfast Spot,Vietnamese Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Clothing Store,Middle Eastern Restaurant,Thai Restaurant,Noodle House,Italian Restaurant
1,Studio District,Pizza Place,Fast Food Restaurant,Pharmacy,Coffee Shop,Chinese Restaurant,Grocery Store,Sandwich Place,Bank,Playground,Asian Restaurant
2,The Beaches,History Museum,Fast Food Restaurant,Bar,Construction & Landscaping,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store
3,"The Danforth West , Riverdale",Bakery,Coffee Shop,Indian Restaurant,Skating Rink,Bus Line,Playground,Bank,Hakka Restaurant,Chinese Restaurant,Intersection


#### 04. Cluster Neighbourhood

In [171]:
# set number of clusters - Only four clusters
kclusters = 4

Toronto_grouped_clustering =Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1, 3], dtype=int32)

In [182]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1.0,History Museum,Fast Food Restaurant,Bar,Construction & Landscaping,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,3.0,Bakery,Coffee Shop,Indian Restaurant,Skating Rink,Bus Line,Playground,Bank,Hakka Restaurant,Chinese Restaurant,Intersection
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572,0.0,Breakfast Spot,Vietnamese Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Clothing Store,Middle Eastern Restaurant,Thai Restaurant,Noodle House,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2.0,Pizza Place,Fast Food Restaurant,Pharmacy,Coffee Shop,Chinese Restaurant,Grocery Store,Sandwich Place,Bank,Playground,Asian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
Toronto_merged.dtypes

PostalCode                 object
Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [184]:
Toronto_merged=Toronto_merged.dropna()

Converting float values into Integer

In [185]:
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype(int)

In [186]:
Toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,History Museum,Fast Food Restaurant,Bar,Construction & Landscaping,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,3,Bakery,Coffee Shop,Indian Restaurant,Skating Rink,Bus Line,Playground,Bank,Hakka Restaurant,Chinese Restaurant,Intersection
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572,0,Breakfast Spot,Vietnamese Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Clothing Store,Middle Eastern Restaurant,Thai Restaurant,Noodle House,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Pizza Place,Fast Food Restaurant,Pharmacy,Coffee Shop,Chinese Restaurant,Grocery Store,Sandwich Place,Bank,Playground,Asian Restaurant


In [187]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters